# binder-profiling
Read your .csv file and give you better insight of your data using [Pandas Profiling](pandas-profiling). Please consider to visit and leave a star on [binder-profiling](https://github.com/yasirroni/binder-profiling) if you like this repository.

In [ ]:
from IPython.display import display, HTML, clear_output
from ipywidgets import widgets
from notebook import notebookapp
import pandas as pd
import numpy as np
import csv
import time
import threading
import os

from pandas_profiling import ProfileReport

In [ ]:
# button layout
layout_button = widgets.Layout(
    width = '120px'
)

In [ ]:
# uploader
uploader = widgets.FileUpload(
    accept='.csv*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    )

# label
label_main = widgets.Label(
    value = 'Upload File in .csv Format'
)

# button_show_profiling
button_show = widgets.Button(
    description = 'show',
    button_style = '',
    layout = layout_button,
    disabled = True,
)

# button_json
button_json = widgets.Button(
    description = '.json',
    button_style = '',
    layout = layout_button,
    disabled = True,
)

# button_html
button_html = widgets.Button(
    description = '.html',
    button_style = '',
    layout = layout_button,
    disabled = True,
)

# button_clear
button_clear =  widgets.Button(
    description = 'clear display',
    button_style = '',
    layout = layout_button,
    disabled = True,
)

In [ ]:
def button_show_click(_):
    display(profile)
    button_show.disabled = True
    
button_show.on_click(button_show_click)

def button_json_click(_):
    profile.to_file(file_name+'.json')
    servers = list(notebookapp.list_running_servers())
    html_url = os.path.join('https://notebooks.gesis.org'+str(servers[0]['base_url']),'view',file_name+'.json')
    html_text = '<a href="' + html_url +'"' + 'target="_blank">' + html_url + '</a>'
    display(HTML(html_text))
    button_json.disabled = True

button_json.on_click(button_json_click)

def button_html_click(_):
    profile.to_file(file_name+'.html')
    servers = list(notebookapp.list_running_servers())
    html_url = os.path.join('https://notebooks.gesis.org'+str(servers[0]['base_url']),'view',file_name+'.html')
    html_text = '<a href="' + html_url +'"' + 'target="_blank">' + html_url + '</a>'
    display(HTML(html_text))
    button_html.disabled = True

button_html.on_click(button_html_click)

def button_clear_click(_):
    clear_output()
    if uploader.value:
        button_show.disabled = False
        button_json.disabled = False
        button_html.disabled = False
    display(button_box)

button_clear.on_click(button_clear_click)

In [ ]:
# button box wrapper
button_box = widgets.HBox(
    children = (button_show, button_json, button_html, button_clear)
)

# wrapper
wrapper = widgets.VBox(
    children=(uploader, label_main)
)

# outputs box wrapper
profile_box = widgets.VBox(
    children = ()
)

In [ ]:
# get content and name
def get_content_and_name(uploader):
    for key, value in uploader.value.items():
        return value['content'], value['metadata']['name']

In [ ]:
def write_file_from_bytes_data(bytes_data, file_full_name='temp.csv'):
    with open(file_full_name,'wb') as f:
        f.write(bytes_data)

In [ ]:
def work():
    # global flag
    global thread_status, file_name, profile
    
    # save uploader.value for state reference
    current_uploader_content = uploader.value
    
    while thread_status:
        
        # check if there is a change in uploader.value
        if current_uploader_content != uploader.value:
            
            # renew state reference
            current_uploader_content = uploader.value
           
            if uploader.value:
                # message
                label_main.value = 'Reading Your File...'
                
                # get content and file name
                content, file_full_name = get_content_and_name(uploader)
                file_name = file_full_name.split('.')[0]
                
                # from bytes data to csv file
                write_file_from_bytes_data(content, file_full_name=file_full_name)
                
                # pandas read csv
                df = pd.read_csv(file_full_name)
                
                label_main.value = 'Profiling Your File...'
                
                # generate the profiling report
                profile = ProfileReport(df, title=file_name, html={'style': {'full_width': True}}, sort="None")
                
                # update label
                label_main.value = 'Profiling Finished! Click Below For Show or Export'
                
                # export button
                button_show.disabled = False
                button_json.disabled = False
                button_html.disabled = False
                button_clear.disabled = False
                
        # sleep for threading
        time.sleep(0.05)
        
        # exit flag from threading
        if thread_status == False:
            break

In [ ]:
# thread status
thread_status = False
thread_status = True

# create thread
thread = threading.Thread(target=work)

# start thread
thread.start()

In [ ]:
display(wrapper)

In [ ]:
display(button_box)